In [1]:
#import tuxml
import pandas as pd 
import warnings
warnings.filterwarnings('ignore')

#df = tuxml.load_dataset()
# path = r"x264/sampledConfigurations_distBased_t3.csv"
#path = r"sampledConfigurations_distBased_t3.csv"
samples_config = ['distBased', 'divDistBased','henard', 'random', 'solverBased', 'twise']

# for config_name in samples_config:
#     path = r"sampledConfigurations_"+config_name+"_t3.csv"
# #     df.append(pd.read_csv(path, sep=';'))
#     display(path)

# df = pd.read_csv(path, sep=';')


In [2]:
# df_importance = pd.read_csv("feature_importanceRF.csv")
# df_importance = df[df_importance[:100]["Unnamed: 0"].values]
# df_importance.head()

In [2]:
#size_methods = ["vmlinux", "GZIP-bzImage", "GZIP-vmlinux", "GZIP", "BZIP2-bzImage", 
#              "BZIP2-vmlinux", "BZIP2", "LZMA-bzImage", "LZMA-vmlinux", "LZMA", "XZ-bzImage", "XZ-vmlinux", "XZ", 
#?              "LZO-bzImage", "LZO-vmlinux", "LZO", "LZ4-bzImage", "LZ4-vmlinux", "LZ4"]

size_methods = ["Performance"]
#additional_ft = ["nbyes", "nbno", "nbmodule", "nbyesmodule"]

In [3]:
def features_engine(df):
    
    engine_values = []
    for i, j in df.iterrows():
        num1s = 0
        num1s = j.value_counts()[1]
        engine_values.append(num1s)
    
    df['construction'] = engine_values
    

In [4]:
path = r"sampledConfigurations_distBased_t3.csv"
df = pd.read_csv(path, sep=';')
features_engine(df)
display(df)

,root,API,OpenCL,CUDA,TextureMemory,Linear1D,Linear2D,Array2D,LocalMemory,Blocksize,...,padding_448,padding_480,padding_512,pixelPerThread,pixelPerThread_1,pixelPerThread_2,pixelPerThread_3,pixelPerThread_4,Performance,construction
0,1,1,1,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,1,21.173,9
1,1,1,1,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,1,21.185,9
2,1,1,1,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,1,21.185,9
3,1,1,1,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,1,21.211,9
4,1,1,1,0,0,0,0,0,0,1,...,0,0,1,1,0,0,0,1,21.226,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,1,1,0,1,0,0,0,0,1,1,...,0,0,0,1,1,0,0,0,105.101,10
4597,1,1,0,1,0,0,0,0,1,1,...,0,0,0,1,1,0,0,0,106.481,10
4598,1,1,1,0,1,0,0,1,1,1,...,0,0,0,1,0,1,0,0,107.720,12
4599,1,1,1,0,0,0,0,0,1,1,...,0,0,0,1,1,0,0,0,112.250,10


In [5]:
#learning a model by using linear regression
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import ElasticNet

def run_regressorML(reg, test_size, size_target, x_train, x_test):
    assert(size_target in size_methods)
    reg.fit(x_train, y_train)
    y_pred = reg.predict(x_test)
    #y_pred = reg.intercept_ + np.sum(reg.coef_ * x_test.values, axis=1)
    dfErrors = pd.DataFrame({'Actual':y_test, 'Predicted':y_pred, "error":(y_pred - y_test).abs(), "% error":((y_pred - y_test)/y_test).abs()*100})
    return dfErrors["% error"].describe()

In [6]:
def ft_importances(coef_, col):
    importanceSeries = pd.Series(coef_, index=col.values)
    return importanceSeries[importanceSeries != 0].abs().sort_values(ascending = False)

In [7]:
df_result  = pd.DataFrame(columns = ['algorithm_name', 'ft_dummyfication', 'origin_ft_selection', 'ft_selection', 'hyperparameters', 'size_target', 'test_size', 'coef_order', 'accuracy'])
#df_result  = pd.DataFrame(columns = ['algorithm_name', 'hyperparameters', 'size_target', 'additional_ft', 'test_size', 'coef_order', 'accuracy'])

In [8]:
df_result

,algorithm_name,ft_dummyfication,origin_ft_selection,ft_selection,hyperparameters,size_target,test_size,coef_order,accuracy


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import ElasticNet
from sklearn import linear_model
from sklearn import metrics

ft_importance_enable = True
# def runFeatureSelection(df, config_name, num_features): 
for config_name in samples_config:
    for samplePerc in [0.3,0.4,0.5,0.6]:
        for t in [1,2,3]:
            
            path_name = config_name+"_t"+str(t)
            path = r"sampledConfigurations_"+path_name+".csv"
            
            df = pd.read_csv(path, sep=';')
            num_features = round(samplePerc*len(df.columns))

            df_importance = pd.read_csv("feature_importance_"+path_name+".csv")
            df_importance = df[df_importance[:num_features]["Unnamed: 0"].values]
            
            print("DF with Feature Importance")
            print("Sample features number: "+ str(num_features))
            display(df_importance)
            df_importance.head()

            target = "Performance"
            paramsLasso = {'alpha': [25,10,4,2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.02,0.01],
                           'max_iter': [1, 5, 10, 100, 1000],}
            paramsRidge = {'alpha': [25,10,4,2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.02,0.01]}
            paramsENet = {'alpha': [25,10,4,2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.02,0.01],
                          'l1_ratio': [0.0, 0.5, 1.0, 0.1],
                          'max_iter': [1, 5, 10, 100, 1000],}

            for ft_dummyfication in [False, True]:
                if ft_dummyfication:
                    #Creating dummy variables in pandas
                    df = pd.get_dummies(df, columns=list(df.drop(columns=size_methods).columns.values))
#                     df_importance = pd.get_dummies(df_importance, columns=list(df_importance.drop(columns=size_methods).columns.values))
                    df_importance = pd.get_dummies(df_importance, columns=list(df_importance.columns.values))

                if ft_importance_enable:
                    ft_selection =  {'None': df.drop(columns=size_methods).columns,
                                'RandomForest': df_importance.columns[:num_features],}
                    
                else:
                    ft_selection =  {'None': df.drop(columns=size_methods).columns,
                                     'RandomForest': df.columns[:num_features],}

                for origin_ft_selection in ft_selection:
                    for size in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:

                        x_train, x_test, y_train, y_test = train_test_split(df[ft_selection[origin_ft_selection]], df[target], test_size=size, random_state=42)  
                        x_train, y_train = np.array(x_train.values), np.array(y_train.values)
                        
                        print(x_train)
                        print(y_train)
                        
                        if len(x_train) > 3:

                            model = {'LinearRegression': LinearRegression(),
                                     'Lasso': GridSearchCV(linear_model.Lasso(), param_grid=paramsLasso, cv=3).fit(x_train, y_train).best_estimator_,
                                     'Ridge': GridSearchCV(linear_model.Ridge(), param_grid=paramsRidge, cv=3).fit(x_train, y_train).best_estimator_,
                                     'ElasticNet': GridSearchCV(ElasticNet(), param_grid=paramsENet, scoring='r2', cv=3).fit(x_train, y_train).best_estimator_,
                                     'PolynomialRegression': LinearRegression()}
                        else:
                            if len(x_train) >= 2:
                                model = {'LinearRegression': LinearRegression(),
                                         'Lasso': GridSearchCV(linear_model.Lasso(), param_grid=paramsLasso, cv=2).fit(x_train, y_train).best_estimator_,
                                         'Ridge': GridSearchCV(linear_model.Ridge(), param_grid=paramsRidge, cv=2).fit(x_train, y_train).best_estimator_,
                                         'ElasticNet': GridSearchCV(ElasticNet(), param_grid=paramsENet, scoring='r2', cv=2).fit(x_train, y_train).best_estimator_,
                                         'PolynomialRegression': LinearRegression()}
                        for key in model:
                            if (key == 'PolynomialRegression') and (origin_ft_selection == 'None'):
                                print("It doesn't scale")
                            else:
                                if (key == 'PolynomialRegression'):
                                    #for 2 options we have: 𝑓(𝑥₁, 𝑥₂) = 𝑏₀ + 𝑏₁𝑥₁ + 𝑏₂𝑥₂ + 𝑏₃𝑥₁² + 𝑏₄𝑥₁𝑥₂ + 𝑏₅𝑥₂²
                                    x_train = PolynomialFeatures(degree=2, include_bias=False).fit_transform(x_train)
                                    x_test = PolynomialFeatures(degree=2, include_bias=False).fit_transform(x_test)

                                res_model = run_regressorML(model[key], test_size=size, size_target=target, x_train=x_train, x_test=x_test)

                                if (key == 'PolynomialRegression'):
                                    model[key].coef_ = model[key].coef_[:num_features]

#                                 print(model[key].coef_)
                                coef_order = ft_importances(model[key].coef_, col=ft_selection[origin_ft_selection])
                                df_result.loc[len(df_result)] = [key, ft_dummyfication, origin_ft_selection, ft_selection[origin_ft_selection], model[key], target, size, coef_order, res_model]
            display(df_result)
            if ft_importance_enable:
                df_result.to_csv("results_with_feature_importance_"+path_name+"_"+str(num_features)+".csv", header=True)
            else:
                df_result.to_csv("results_"+path_name+"_"+str(num_features)+".csv", header=True)

DF with Feature Importance
Sample features number: 17


,LocalMemory,bs_32x1,pixelPerThread_1,bs_32x8,bs_512x1,pixelPerThread_4,Linear2D,bs_64x2,bs_32x4,bs_32x2,bs_128x1,bs_64x4,bs_128x2,CUDA,bs_256x2,OpenCL,padding_512
0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0
2,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0
4,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0
5,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0
6,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0
7,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
8,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
9,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0


[[1 1 0 ... 0 0 0]
 [1 1 1 ... 0 0 1]
 [1 1 1 ... 0 0 1]
 ...
 [1 1 1 ... 1 0 0]
 [1 1 1 ... 0 0 1]
 [1 1 1 ... 0 0 1]]
[65.124 36.89  35.332 50.264 23.299 22.406 21.617 52.176 22.386 21.527
 22.274 58.193 64.835 64.996 22.591 22.403 22.629 35.225 50.538 50.255
 21.279 64.594 37.022 50.275 22.073 42.933 22.404 51.774 21.458 25.114
 21.463 64.509 51.379 35.012 58.067 22.403 26.954 23.159 77.907 24.194
 22.342 63.281 22.573 37.755 55.102]
It doesn't scale
[[1 1 0 ... 0 1 0]
 [1 1 0 ... 0 0 1]
 [1 1 0 ... 0 0 1]
 ...
 [1 1 1 ... 1 0 0]
 [1 1 1 ... 0 0 1]
 [1 1 1 ... 0 0 1]]
[22.406 21.617 52.176 22.386 21.527 22.274 58.193 64.835 64.996 22.591
 22.403 22.629 35.225 50.538 50.255 21.279 64.594 37.022 50.275 22.073
 42.933 22.404 51.774 21.458 25.114 21.463 64.509 51.379 35.012 58.067
 22.403 26.954 23.159 77.907 24.194 22.342 63.281 22.573 37.755 55.102]
It doesn't scale
[[1 1 0 ... 1 0 0]
 [1 1 1 ... 0 0 1]
 [1 1 0 ... 0 0 0]
 ...
 [1 1 1 ... 1 0 0]
 [1 1 1 ... 0 0 1]
 [1 1 1 ... 0 0 1]]


[[0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0]
 [1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1]
 [1 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0]
 [1 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 1]
 [1 0 0 0 1 1 1 0 0 0 0 0 0 1 0 0 0]
 [1 0 0 0 1 1 1 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0]
 [1 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 1]
 [1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1]
 [1 0 0 0 1 1 1 0 0 0 0 0 0 1 0 0 1]
 [0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0]
 [1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 1]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0]
 [1 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0]
 [1 0 0 0 1 1 1 0 0 0 0 0 0 1 0 0 0]
 [1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 1]
 [1 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 1]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0]
 

It doesn't scale
[[1 1 1 ... 0 0 1]
 [1 1 1 ... 0 1 0]
 [1 1 1 ... 0 0 1]
 ...
 [1 1 0 ... 0 1 0]
 [1 1 0 ... 0 0 1]
 [1 1 0 ... 0 0 1]]
[21.463 64.509 51.379 35.012 58.067 22.403 26.954 23.159 77.907 24.194
 22.342 63.281 22.573 37.755 55.102]
It doesn't scale
[[1 1 1 0 0 1 1 0 1 0 1 0 1 1 0 1 1 0 0 1 1 1 1 1 0 1 1 0 1 0 1 0 1 1 1 0
  1 1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 1 0 1 1 0 1 0 1 0 1 0 1 0 1 0 1 1 1 0
  1 0 1 1 1 0 1 0 1 0 1 1 0 1 0 0 1 1 0]
 [1 1 0 1 1 0 1 0 1 0 1 0 1 1 0 1 1 0 0 1 1 1 1 1 0 1 1 0 1 0 1 0 1 1 1 0
  1 1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 1 1 0 1 0 1 0 1 0 1 0 1 0 1 1 1 0
  1 1 0 1 1 0 1 0 1 0 1 0 1 1 0 1 0 1 0]
 [1 1 1 0 0 1 1 0 1 0 1 0 1 1 0 1 1 0 0 1 1 1 1 1 0 1 1 0 1 0 1 0 1 1 1 0
  1 1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 1 0 1 1 0 0 1 1 0 1 0 1 0 1 0 1 1 1 0
  1 1 0 1 1 0 1 0 1 0 1 1 0 1 0 0 1 1 0]
 [1 1 0 1 1 0 1 0 1 0 1 0 1 1 0 1 0 1 1 0 1 1 1 1 0 1 1 0 1 0 1 0 1 1 1 0
  1 1 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 1 1 0 1 0 1 0 1 0 1 0 1 0 1 1 1 0
  1 1 0 1 1 0 1 0 1 0 1

...

,algorithm_name,ft_dummyfication,origin_ft_selection,ft_selection,hyperparameters,size_target,test_size,coef_order,accuracy
0,LinearRegression,False,None,"Index(['root', 'API', 'OpenCL', 'CUDA', 'Textu...",LinearRegression(),Performance,0.1,root 1.437513e+15 API ...,count 5.000000e+00 mean 2.107497e+12 st...
1,Lasso,False,None,"Index(['root', 'API', 'OpenCL', 'CUDA', 'Textu...","Lasso(alpha=0.8, max_iter=100)",Performance,0.1,LocalMemory 14.904767 bs_512x1 ...,count 5.000000 mean 10.223368 std ...
2,Ridge,False,None,"Index(['root', 'API', 'OpenCL', 'CUDA', 'Textu...",Ridge(),Performance,0.1,bs_32x8 12.887460 bs_32x1 ...,count 5.000000 mean 8.841700 std ...
3,ElasticNet,False,None,"Index(['root', 'API', 'OpenCL', 'CUDA', 'Textu...","ElasticNet(alpha=0.05, l1_ratio=0.1, max_iter=10)",Performance,0.1,bs_32x8 11.201784 bs_512x1 ...,count 5.000000 mean 7.981183 std ...
4,LinearRegression,False,None,"Index(['root', 'API', 'OpenCL', 'CUDA', 'Textu...",LinearRegression(),Performance,0.2,Array2D 1.566147e+15 Ldg ...,count 1.000000e+01 mean 3.896756e+14 st...
...,...,...,...,...,...,...,...,...,...
157,LinearRegression,True,RandomForest,"Index(['LocalMemory_0', 'LocalMemory_1', 'bs_3...",LinearRegression(),Performance,0.9,bs_32x8_0 8.673500e+00 bs_32x8_1 ...,count 45.000000 mean 31.822487 std ...
158,Lasso,True,RandomForest,"Index(['LocalMemory_0', 'LocalMemory_1', 'bs_3...","Lasso(alpha=25, max_iter=1)",Performance,0.9,"Series([], dtype: float64)",count 45.000000 mean 49.541122 std ...
159,Ridge,True,RandomForest,"Index(['LocalMemory_0', 'LocalMemory_1', 'bs_3...",Ridge(alpha=25),Performance,0.9,bs_32x8_0 1.223673 bs_32x8_1 ...,count 45.000000 mean 42.843958 std ...
160,ElasticNet,True,RandomForest,"Index(['LocalMemory_0', 'LocalMemory_1', 'bs_3...","ElasticNet(alpha=25, l1_ratio=0.0, max_iter=1)",Performance,0.9,bs_32x8_0 0.297257 bs_32x8_1 ...,count 45.000000 mean 47.880988 std ...


DF with Feature Importance
Sample features number: 17


,LocalMemory,bs_1024x1,bs_512x1,bs_32x1,pixelPerThread_1,pixelPerThread_2,Linear1D,bs_256x4,bs_32x32,TextureMemory,bs_128x8,bs_512x2,bs_64x16,bs_32x8,pixelPerThread_4,bs_256x1,pixelPerThread_3
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
838,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
839,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
840,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
841,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


[[1 1 0 ... 0 0 1]
 [1 1 1 ... 0 0 0]
 [1 1 0 ... 0 0 1]
 ...
 [1 1 0 ... 1 0 0]
 [1 1 0 ... 0 1 0]
 [1 1 0 ... 1 0 0]]
[ 50.252  26.954  64.347  39.288  55.102  22.638  22.175  22.24   37.575
  38.806  48.957  44.674  23.31   43.352  39.456  23.285  33.122  24.468
  26.146  22.84   48.571  26.049  24.248  46.678  23.077  21.311  21.923
  64.648  71.276  23.807  36.848  44.462  22.345  50.275  22.23   38.803
  54.52   75.554  25.254  53.011  34.311  35.901  22.268  21.422  50.538
  23.834  62.301  22.467  74.863  23.641  21.642  68.634  27.838  63.712
  50.04   22.259  63.414  25.924  38.588  42.524  31.419  57.545  73.685
  51.774  33.167  39.411  23.354  23.158  23.751  21.33   48.954  22.756
  26.04   22.357  33.026  33.361  26.058  34.784  21.279  26.957  33.317
  63.642  23.203  66.218  34.973  69.812  38.823  24.025  26.965  74.959
  37.294  80.028  51.794  22.246  22.725  50.318  46.699  29.632  26.575
  23.304  51.285  46.931  22.591  31.409  77.661  38.455  21.929  35.221
  46

It doesn't scale
[[1 1 0 ... 1 0 0]
 [1 1 0 ... 1 0 0]
 [1 1 0 ... 0 0 0]
 ...
 [1 1 0 ... 1 0 0]
 [1 1 0 ... 0 1 0]
 [1 1 0 ... 1 0 0]]
[ 42.615  24.367  64.668  22.677  23.691  23.408  50.667  33.092  22.575
  42.388  44.742  86.617  21.8    37.954  22.403  40.803  21.934  32.895
  24.936  72.235  21.568  33.557  28.99   29.767  21.887  28.001  65.133
  24.015  35.79   42.578  38.816  53.094  21.527  44.316  24.48   22.267
  21.96   36.982  24.411  25.28   31.355  35.715  23.008  26.229  38.421
  23.444  34.144  73.357  26.397  93.884  35.488  37.907  22.92   41.897
  64.92   22.791  23.823  39.023  63.425  22.2    65.392  33.763  33.135
  22.398  22.421  34.897  22.324  33.21   22.321  44.685  65.148  69.674
  37.876  38.248  22.324  21.458  63.281  21.823  21.801  26.085  22.833
  62.325  37.465  47.617  33.28   22.932  51.905  46.164  22.926  42.24
  58.504  42.653  23.895  37.047  26.215  41.854  21.482  21.299  27.804
  23.888  41.851  32.054  27.864  23.438  65.063  41.021  32.

In [ ]:
len(x_train)

In [ ]:
df_result['accuracy'][0]

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

rs = []
for i in range(len(df_result['accuracy'])):
    rs.append(df_result['accuracy'][i]['mean'])

# rs
# display (df_result['algorithm_name'], df_result['accuracy'], rs)

plt.scatter(df_result['algorithm_name'], rs, color ="blue", marker = "s")
plt.plot(df_result['algorithm_name'], rs, color ="blue")


In [ ]:
# Boxplot
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


data = [model['LinearRegression'].coef_, model['Lasso'].coef_, model['Ridge'].coef_, model['ElasticNet'].coef_, model['PolynomialRegression'].coef_]
fig1, ax1 = plt.subplots()
ax1.set_title('Coeficient of feature importance for all algorithms')
ax1.boxplot(data)
plt.show()